In [10]:
'''
Overview
this notebook parses all past ufc fight stats when run, it does not include upcoming fights


scrape ufc fight stats
get all event details, name, url, date, location for all ufc events
for each event, get fight details all fights on card
parse each fight to get fight stats of both fighters
'''


'\nOverview\nthis notebook parses all past ufc fight stats when run, it does not include upcoming fights\n\n\nscrape ufc fight stats\nget all event details, name, url, date, location for all ufc events\nfor each event, get fight details all fights on card\nparse each fight to get fight stats of both fighters\n'

In [ ]:
# imports
import pandas as pd
from tqdm.notebook import tqdm_notebook

# import library
import scripts_ufc.config.scrape_ufc_stats_library as LIB

# import config
import yaml
config = yaml.safe_load(open('scrape_ufc_stats_config.yaml'))

Parse Event Details
Includes:
Event
URL
Date
Location

In [12]:
# define url to parse
events_url = config['completed_events_all_url']

In [13]:
# get soup
soup = LIB.get_soup(events_url)

# parse event details
all_event_details_df = LIB.parse_event_details(soup)

# show event details
display(all_event_details_df)

# write event details to file
all_event_details_df.to_csv(config['event_details_file_name'], index=False)

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Bonfim vs. Brown,http://ufcstats.com/event-details/6436029b50a9...,"November 08, 2025","Las Vegas, Nevada, USA"
1,UFC Fight Night: Garcia vs. Onama,http://ufcstats.com/event-details/0e2c2daf11b5...,"November 01, 2025","Las Vegas, Nevada, USA"
2,UFC 321: Aspinall vs. Gane,http://ufcstats.com/event-details/7956f026e267...,"October 25, 2025","Abu Dhabi, Abu Dhabi, United Arab Emirates"
3,UFC Fight Night: De Ridder vs. Allen,http://ufcstats.com/event-details/38e5d9dcb0fd...,"October 18, 2025","Vancouver, British Columbia, Canada"
4,UFC Fight Night: Oliveira vs. Gamrot,http://ufcstats.com/event-details/9336e86cfd4c...,"October 11, 2025","Rio de Janeiro, Rio de Janeiro, Brazil"
...,...,...,...,...
748,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, Wyoming, USA"
749,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, North Carolina, USA"
750,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, Oklahoma, USA"
751,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, North Carolina, USA"


Parse Fight Details
Includes:
Event
Bout
URL

In [14]:
# define list of urls of fights to parse
list_of_events_urls = list(all_event_details_df['URL'])

In [15]:
# create empty df to store fight details
all_fight_details_df = pd.DataFrame(columns=config['fight_details_column_names'])

# loop through each event and parse fight details
for url in tqdm_notebook(list_of_events_urls):

    # get soup
    soup = LIB.get_soup(url)

    # parse fight links
    fight_details_df = LIB.parse_fight_details(soup)
    
    # concat fight details
    all_fight_details_df = pd.concat([all_fight_details_df, fight_details_df])

# show all fight details
display(all_fight_details_df)

# write fight details to file
all_fight_details_df.to_csv(config['fight_details_file_name'], index=False)

  0%|          | 0/753 [00:00<?, ?it/s]

,EVENT,BOUT,URL
0,UFC Fight Night: Bonfim vs. Brown,Gabriel Bonfim vs. Randy Brown,http://ufcstats.com/fight-details/4e224ebb6394...
1,UFC Fight Night: Bonfim vs. Brown,Joseph Morales vs. Matt Schnell,http://ufcstats.com/fight-details/f7e9bd2d6f81...
2,UFC Fight Night: Bonfim vs. Brown,Uros Medic vs. Muslim Salikhov,http://ufcstats.com/fight-details/b23fa678cc65...
3,UFC Fight Night: Bonfim vs. Brown,Chris Padilla vs. Ismael Bonfim,http://ufcstats.com/fight-details/20971ed9a842...
4,UFC Fight Night: Bonfim vs. Brown,Christian Leroy Duncan vs. Marco Tulio,http://ufcstats.com/fight-details/ac4343150638...
...,...,...,...
10,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,http://ufcstats.com/fight-details/3b020d4914b4...
11,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,http://ufcstats.com/fight-details/d917c8c7461b...
12,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,http://ufcstats.com/fight-details/ccee020be2e8...
13,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,http://ufcstats.com/fight-details/4b9ae533ccb3...


Parse Fight Results and Fight Stats
Fight Results Includes:
Event
Bout
Weightclass
Method
Round
Time
Time Format
Referee
Details

Fight Stats Includes:
Event
Bout
Round
Fighter
Kd
Sig.Str.
Sig.Str. %
Total Str.
Td
Td %
Sub.Att
Rev.
Ctrl
Head
Body
Leg
Distance
Clinch
Ground

In [16]:
# define list of urls of fights to parse
list_of_fight_details_urls = list(all_fight_details_df['URL'])

In [17]:
# create empty df to store fight results
all_fight_results_df = pd.DataFrame(columns=config['fight_results_column_names'])
# create empty df to store fight stats
all_fight_stats_df = pd.DataFrame(columns=config['fight_stats_column_names'])

# loop through each fight and parse fight results and stats
for url in tqdm_notebook(list_of_fight_details_urls):

    # get soup
    soup = LIB.get_soup(url)

    # parse fight results and fight stats
    fight_results_df, fight_stats_df = LIB.parse_organise_fight_results_and_stats(
        soup,
        url,
        config['fight_results_column_names'],
        config['totals_column_names'],
        config['significant_strikes_column_names']
        )

    # concat fight results
    all_fight_results_df = pd.concat([all_fight_results_df, fight_results_df])
    # concat fight stats
    all_fight_stats_df = pd.concat([all_fight_stats_df, fight_stats_df])

# show all fight results
display(all_fight_results_df)
# show all fight stats
display(all_fight_stats_df)

# write to file
all_fight_results_df.to_csv(config['fight_results_file_name'], index=False)
# write to file
all_fight_stats_df.to_csv(config['fight_stats_file_name'], index=False)

  0%|          | 0/8440 [00:00<?, ?it/s]

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC Fight Night: Bonfim vs. Brown,Gabriel Bonfim vs. Randy Brown,W/L,Welterweight Bout,KO/TKO,2,1:40,5 Rnd (5-5-5-5-5),Mark Smith,Knee to Head In Clinch,http://ufcstats.com/fight-details/4e224ebb6394...
0,UFC Fight Night: Bonfim vs. Brown,Matt Schnell vs. Joseph Morales,L/W,Flyweight Bout,Submission,1,2:54,3 Rnd (5-5-5),Keith Peterson,Guillotine Choke From Guard,http://ufcstats.com/fight-details/f7e9bd2d6f81...
0,UFC Fight Night: Bonfim vs. Brown,Muslim Salikhov vs. Uros Medic,L/W,Welterweight Bout,KO/TKO,1,1:03,3 Rnd (5-5-5),Kerry Hatley,Punch to Head At Distance,http://ufcstats.com/fight-details/b23fa678cc65...
0,UFC Fight Night: Bonfim vs. Brown,Chris Padilla vs. Ismael Bonfim,W/L,Lightweight Bout,KO/TKO,2,4:30,3 Rnd (5-5-5),Chris Tognoni,Punches to Head On Ground,http://ufcstats.com/fight-details/20971ed9a842...
0,UFC Fight Night: Bonfim vs. Brown,Christian Leroy Duncan vs. Marco Tulio,W/L,Middleweight Bout,KO/TKO,2,3:20,3 Rnd (5-5-5),Mark Smith,Punch to Head At Distance,http://ufcstats.com/fight-details/ac4343150638...
...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...


,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC Fight Night: Bonfim vs. Brown,Gabriel Bonfim vs. Randy Brown,Round 1,Gabriel Bonfim,0,24 of 43,55%,24 of 43,0 of 0,---,0,0,0:00,6 of 21,0 of 0,18 of 22,23 of 42,1 of 1,0 of 0
1,UFC Fight Night: Bonfim vs. Brown,Gabriel Bonfim vs. Randy Brown,Round 2,Gabriel Bonfim,1,11 of 21,52%,11 of 21,0 of 0,---,0,0,0:00,5 of 15,1 of 1,5 of 5,10 of 20,1 of 1,0 of 0
0,UFC Fight Night: Bonfim vs. Brown,Gabriel Bonfim vs. Randy Brown,Round 1,Randy Brown,0,19 of 61,31%,19 of 61,0 of 0,---,0,0,0:00,7 of 33,3 of 14,9 of 14,19 of 61,0 of 0,0 of 0
1,UFC Fight Night: Bonfim vs. Brown,Gabriel Bonfim vs. Randy Brown,Round 2,Randy Brown,0,14 of 26,53%,14 of 26,0 of 0,---,0,0,0:00,4 of 15,6 of 7,4 of 4,14 of 26,0 of 0,0 of 0
0,UFC Fight Night: Bonfim vs. Brown,Matt Schnell vs. Joseph Morales,Round 1,Matt Schnell,0,4 of 11,36%,5 of 12,1 of 1,100%,0,1,0:15,1 of 8,2 of 2,1 of 1,4 of 11,0 of 0,0 of 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0,4 of 5,80%,95 of 102,0 of 0,---,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0,1 of 1,100%,1 of 1,0 of 1,0%,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0,1 of 1,100%,2 of 2,0 of 0,---,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0
0,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0,1 of 1,100%,2 of 2,1 of 1,100%,1,0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0
